In [74]:
from pyspark.sql import SparkSession
#import pandas as pd
import numpy as np
from pyspark.sql.functions import *
#import boto3
import io
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
plyr=spark.read.format("csv").load("s3://kalange-european-soccer/ply.csv",header='true')
coun = spark.read.format("csv").load("s3://kalange-european-soccer/country.csv",header='true')
league = spark.read.format("csv").load("s3://kalange-european-soccer/league.csv",header='true')
#match = spark.read.format("csv").load("s3://kalange-european-soccer/match1.csv",header='true')

team_attr = spark.read.format("csv").load("s3://kalange-european-soccer/team_attr.csv",header='true')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
team = spark.read.format("csv").load("s3://kalange-european-soccer/team.csv",header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
plyr_attr= spark.read.format("csv").load("s3://kalange-european-soccer/ply_attr.csv",header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
xply=plyr_attr.select(plyr_attr.columns[:7])
yply=plyr_attr.select(['id','stamina','strength'])

xply=xply.alias('xply')
yply=yply.alias('yply')

df_plyr=xply.join(yply,xply.id==yply.id).select("xply.id","player_fifa_api_id","player_api_id","date","overall_rating", \
                                               "potential","stamina","strength")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_plyr.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+-------------+-------------------+--------------+---------+-------+--------+
| id|player_fifa_api_id|player_api_id|               date|overall_rating|potential|stamina|strength|
+---+------------------+-------------+-------------------+--------------+---------+-------+--------+
|  1|            218353|       505942|2016-02-18 00:00:00|          67.0|     71.0|   54.0|    76.0|
+---+------------------+-------------+-------------------+--------------+---------+-------+--------+
only showing top 1 row

In [7]:
#plyr.select([count(when(isnan(c), c)).alias(c) for c in plyr.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#coun.select([count(when(isnan(c), c)).alias(c) for c in coun.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
match= spark.read.format("csv").load("s3://kalange-european-soccer/match.csv",header='true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
mat=match.select(['id','country_id','league_id','season','stage','date','match_api_id','home_team_api_id','away_team_api_id',\
                'home_team_goal','away_team_goal'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#mat.select([count(when(isnan(c), c)).alias(c) for c in mat.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
"""
Dropping NaN values for the player attributes
"""


playr=df_plyr.na.drop(subset=["overall_rating"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
time = mat.select(  
                    date_format(col('date'),"yyyy-MM-dd").alias('date'),
                    dayofmonth('date').alias('day'),
                    weekofyear('date').alias('week'),
                    month('date').alias('month'),
                    year('date').alias('year'),
                    date_format('date','E').alias('weekday')).dropDuplicates()  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
matches=mat.withColumn("match_date",date_format(col('date'),"yyyy-MM-dd").cast("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
time=time.withColumn("date",col('date').cast("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
coun=coun.withColumnRenamed("id","country_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
team.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: string (nullable = true)
 |-- id: string (nullable = true)
 |-- team_api_id: string (nullable = true)
 |-- team_fifa_api_id: string (nullable = true)
 |-- team_long_name: string (nullable = true)
 |-- team_short_name: string (nullable = true)

In [94]:
team=team.withColumnRenamed("team_api_id","team_id")
team=team.withColumnRenamed("team_long_name","team_name")
team=team.select("team_id","team_name","team_short_name")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
time=time.withColumn('time_id',monotonically_increasing_id())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
matches=matches.withColumn('Total_Goals',col("home_team_goal")+col("away_team_goal"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
matches=matches.withColumnRenamed("home_team_api_id","home_team")
matches=matches.withColumnRenamed("away_team_api_id","away_team")
matches=matches.withColumnRenamed("home_team_goal","home_goal")
matches=matches.withColumnRenamed("away_team_goal","away_goal")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
matches.show(2)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1596735674651_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'matches' is not defined
Traceback (most recent call last):
NameError: name 'matches' is not defined



In [21]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
def winning(home_t,away_t,home_g,away_g):
    if home_g>away_g:
        return home_t
    if away_g>home_g:
        return away_t
    else :
        return "draw" 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
udf_win = F.udf(winning,StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
matches=matches.withColumn("Win_team",udf_win("home_team","away_team","home_goal","away_goal"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [101]:
matches.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----------+---------+---------+-----+-------------------+------------+---------+---------+---------+---------+----------+-----------+--------+
| id|country_id|league_id|   season|stage|               date|match_api_id|home_team|away_team|home_goal|away_goal|match_date|Total_Goals|Win_team|
+---+----------+---------+---------+-----+-------------------+------------+---------+---------+---------+---------+----------+-----------+--------+
|  1|         1|        1|2008/2009|    1|2008-08-17 00:00:00|      492473|     9987|     9993|        1|        1|2008-08-17|        2.0|    draw|
|  2|         1|        1|2008/2009|    1|2008-08-16 00:00:00|      492474|    10000|     9994|        0|        0|2008-08-16|        0.0|    draw|
|  3|         1|        1|2008/2009|    1|2008-08-16 00:00:00|      492475|     9984|     8635|        0|        3|2008-08-16|        3.0|    8635|
|  4|         1|        1|2008/2009|    1|2008-08-17 00:00:00|      492476|     9991|     9998|        5|       

In [117]:
time.createOrReplaceTempView("Time")
coun.createOrReplaceTempView("Country")
team.createOrReplaceTempView("Team")
matches.createOrReplaceTempView("Match")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
league.createOrReplaceTempView("League")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
time.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---+----+-----+----+-------+-------+
|      date|day|week|month|year|weekday|time_id|
+----------+---+----+-----+----+-------+-------+
|2015-09-19| 19|  38|    9|2015|    Sat|      0|
|2013-01-23| 23|   4|    1|2013|    Wed|      1|
|2009-09-18| 18|  38|    9|2009|    Fri|      2|
|2012-02-12| 12|   6|    2|2012|    Sun|      3|
|2010-03-14| 14|  10|    3|2010|    Sun|      4|
+----------+---+----+-----+----+-------+-------+
only showing top 5 rows

In [36]:
Fact_Goals = spark.sql("""
Select ti.time_id as timeid,
       ti.day,
       ti.week,
       ti.month,
       ti.year,
       ti.weekday,
       c.name as country_name,
       round(avg(m.Total_Goals),2) as avg_goals
from Match m 
join Time ti
on m.match_date=ti.date
join Country c
on m.country_id=c.country_id
group by country_name,timeid,day,week,month,year,weekday,country_name
order by time_id  
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
Fact_Goals.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8255

In [38]:
Fact_Goals.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---+----+-----+----+-------+------------+---------+
|timeid|day|week|month|year|weekday|country_name|avg_goals|
+------+---+----+-----+----+-------+------------+---------+
|     0| 19|  38|    9|2015|    Sat| Netherlands|      2.5|
|     0| 19|  38|    9|2015|    Sat|    Portugal|      2.0|
|     0| 19|  38|    9|2015|    Sat|     Germany|      1.4|
|     0| 19|  38|    9|2015|    Sat|      France|      2.6|
|     0| 19|  38|    9|2015|    Sat|     England|     2.14|
+------+---+----+-----+----+-------+------------+---------+
only showing top 5 rows

In [131]:
Fact_wins = spark.sql("""
Select m.stage,
       m.league_id,
       l.name as league_name,
       t.team_name as Team ,
       count(Win_team) as Num_wins
from Match m
join team t
on m.Win_team=t.team_id
join league l
on l.country_id=m.country_id
where m.Win_team not like 'draw'
group by m.stage,m.league_id,l.name,Team
order by stage
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
Fact_wins.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---------+--------------------+-------------+--------+
|stage|league_id|         league_name|         Team|Num_wins|
+-----+---------+--------------------+-------------+--------+
|    1|        1|Belgium Jupiler L...|  KV Kortrijk|       2|
|    1|     7809|Germany 1. Bundes...|FC Schalke 04|       3|
|    1|    13274|Netherlands Eredi...| RKC Waalwijk|       1|
|    1|    17642|Portugal Liga ZON...|  CD Nacional|       2|
|    1|    10257|       Italy Serie A|      Palermo|       2|
+-----+---------+--------------------+-------------+--------+
only showing top 5 rows

In [87]:
Fact_wins1 = spark.sql("""
Select m.stage,
       m.league_id,
       m.Win_team
from Match m
where m.Win_team NOT LIKE 'draw'

""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
Fact_wins1.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---------+--------+
|stage|league_id|Win_team|
+-----+---------+--------+
|    1|        1|    8635|
|    1|        1|    9991|
|    1|        1|    9985|
|    1|        1|    9996|
+-----+---------+--------+
only showing top 4 rows

In [24]:
df_plyr.printSchema()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- overall_rating: string (nullable = true)
 |-- potential: string (nullable = true)
 |-- stamina: string (nullable = true)
 |-- strength: string (nullable = true)